In [14]:
import json
import xml.etree.ElementTree as ET
import os

def robust_kml_to_geojson(kml_path, output_path):
    if not os.path.exists(kml_path):
        print(f"❌ 找不到檔案: {kml_path}")
        return

    tree = ET.parse(kml_path)
    root = tree.getroot()
    
    # KML 可能使用的多種命名空間
    namespaces = [
        '{http://www.opengis.net/kml/2.2}',
        '{http://earth.google.com/kml/2.2}',
        '{http://earth.google.com/kml/2.1}'
    ]
    
    features = []

    # 遞迴尋找所有 Placemark
    def find_placemarks(element):
        # 嘗試在當前元素下尋找所有 Placemark (不論命名空間)
        # 使用 wildcard '*' 忽略命名空間前綴
        for placemark in element.findall('.//{*}Placemark'):
            name_node = placemark.find('{*}name')
            name_text = name_node.text if name_node is not None else "未命名景點"
            
            # 尋找座標
            coords_node = placemark.find('.//{*}coordinates')
            if coords_node is not None and coords_node.text:
                coord_str = coords_node.text.strip().split()[0]
                parts = coord_str.split(',')
                if len(parts) >= 2:
                    try:
                        lng = float(parts[0])
                        lat = float(parts[1])
                        features.append({
                            "type": "Feature",
                            "geometry": { "type": "Point", "coordinates": [lng, lat] },
                            "properties": { "name": name_text.strip() }
                        })
                    except ValueError:
                        continue

    find_placemarks(root)

    # 輸出
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump({"type": "FeatureCollection", "features": features}, f, ensure_ascii=False, indent=4)
    
    if len(features) > 0:
        print(f"✅ 成功！這次抓到了 {len(features)} 個景點。")
    else:
        print("⚠️ 警告：依然沒有抓到任何點位。請確認你的 KML 檔內是否有『地標點』而非只有『路線』？")

# 執行
robust_kml_to_geojson('map.kml', 'points.json')

✅ 成功！這次抓到了 7 個景點。
